# Exploring Spark with Pandas


Using pandas examples, convert the analysis to pyspark. This is useful if you discover your data grows too large for your tooling.

The purpose of this notebook is to familiarise yourself you the pyspark API. You are welcome to use the R version of this if you wish. As long as you are able to obtain the correct results. We will be using python in this notebook as it is quite widely used through data science and the community is very large.



#### Firstly, let's get our spark session

In [4]:
!pip install pyspark

from pyspark.sql import SparkSession
!pip install pyspark==3.5.2  # Replace 3.1.2 with the version you need


import pandas as pd
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812363 sha256=04548aa2dd04f0fea02eb628484acbb3ea5278156ebcf8e75cb6b1cc94972bb3
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


### Overview


* Joining two dataframes/data sets
* Simple aggregations
* Persisting

#### JOIN: Pandas

We won't use this more in this notebook, but observe how the joins work.

We what happens if you change from the default inner join to outer joins.

In [5]:
customer_raw = [(1, 'bob', 3462543658686),
           (2, 'rob', 9087567565439),
           (3, 'tim', 5436586999467),
           (4, 'tom', 8349756853250)]

customer_cols = ['id', 'name', 'credit_card_number']



orders_raw = [(1, 'ketchup', 'bob', 1.20),
           (2, 'rutabaga', 'bob', 3.35),
           (3, 'fake vegan meat', 'rob', 13.99),
           (4, 'cheesey poofs', 'tim', 3.99),
           (5, 'ice cream', 'tim', 4.95),
           (6, 'protein powder', 'tom', 49.95)]

orders_cols = ['id', 'product_name', 'customer', 'price']

In [6]:
customer_df = pd.DataFrame(customer_raw, columns=customer_cols)
orders_df = pd.DataFrame(orders_raw, columns=orders_cols)

customer_df

joined_df = pd.merge(customer_df, orders_df, how='inner', left_on='name', right_on='customer')
joined_df

## For self study. What happens if (4, 'tom', 8349756853250) in valuesA becomes (4, 'tod', 8349756853250)
## How do the results change?
## More sensibly; what if customers have not made any orders but we still require them in the result set?

,id_x,name,credit_card_number,id_y,product_name,customer,price
0,1,bob,3462543658686,1,ketchup,bob,1.20
1,1,bob,3462543658686,2,rutabaga,bob,3.35
2,2,rob,9087567565439,3,fake vegan meat,rob,13.99
3,3,tim,5436586999467,4,cheesey poofs,tim,3.99
4,3,tim,5436586999467,5,ice cream,tim,4.95
5,4,tom,8349756853250,6,protein powder,tom,49.95


#### JOIN: Spark

In [7]:
customersDF = spark.createDataFrame(customer_raw, customer_cols)

ordersDF = spark.createDataFrame(orders_raw, orders_cols)

# Show tables
customersDF.show()
ordersDF.show()

+---+----+------------------+
| id|name|credit_card_number|
+---+----+------------------+
|  1| bob|     3462543658686|
|  2| rob|     9087567565439|
|  3| tim|     5436586999467|
|  4| tom|     8349756853250|
+---+----+------------------+

+---+---------------+--------+-----+
| id|   product_name|customer|price|
+---+---------------+--------+-----+
|  1|        ketchup|     bob|  1.2|
|  2|       rutabaga|     bob| 3.35|
|  3|fake vegan meat|     rob|13.99|
|  4|  cheesey poofs|     tim| 3.99|
|  5|      ice cream|     tim| 4.95|
|  6| protein powder|     tom|49.95|
+---+---------------+--------+-----+



In [8]:
joinedDF = customersDF.join(ordersDF, customersDF.name == ordersDF.customer)
joinedDF.show()

+---+----+------------------+---+---------------+--------+-----+
| id|name|credit_card_number| id|   product_name|customer|price|
+---+----+------------------+---+---------------+--------+-----+
|  1| bob|     3462543658686|  1|        ketchup|     bob|  1.2|
|  1| bob|     3462543658686|  2|       rutabaga|     bob| 3.35|
|  2| rob|     9087567565439|  3|fake vegan meat|     rob|13.99|
|  3| tim|     5436586999467|  4|  cheesey poofs|     tim| 3.99|
|  3| tim|     5436586999467|  5|      ice cream|     tim| 4.95|
|  4| tom|     8349756853250|  6| protein powder|     tom|49.95|
+---+----+------------------+---+---------------+--------+-----+



## Simple Aggregations

Now let's explore simple aggregations. You will be using these often when doing exploratory work in big data. Remember, the intention here is that you grow familiar with the way the API works, and how to translate inquiries into that API.

> _How much did each person spend?_

In [9]:
joined_df.groupby('name').agg({"price": ["sum"]})

,price
,sum
name,
bob,4.55
rob,13.99
tim,8.94
tom,49.95


In [10]:
import pyspark.sql.functions as f

joinedDF.groupby('name').agg(f.sum('price').alias('total')).show()

+----+-----------------+
|name|            total|
+----+-----------------+
| bob|             4.55|
| rob|            13.99|
| tim|8.940000000000001|
| tom|            49.95|
+----+-----------------+



Let's use bigger data
  * NYC crash data

In [14]:
# save to the filesystem to prevent another load
# ! curl -o rows.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv


In [19]:
# Download the file
!curl -o rows.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

# Check the file size
!ls -lh rows.csv

import pandas as pd

# Get the number of lines in the file
num_lines = !wc -l < rows.csv
num_lines = int(num_lines[0])

# Calculate the number of rows to skip to get a sample
sample_size = 100000  # Adjust this based on your needs and available memory
skip_ratio = max(1, num_lines // sample_size)

# Read a sample of the data to infer datatypes and check structure
df_sample = pd.read_csv('rows.csv', skiprows=lambda x: x > 0 and x % skip_ratio != 0)

print(df_sample.info())
print("\nSample head:")
print(df_sample.head())

# Now, based on the sample, you can decide how to proceed with loading the full dataset
nyc_df = pd.read_csv('rows.csv')

















  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  427M    0  427M    0     0  4178k      0 --:--:--  0:01:44 --:--:-- 2853k
-rw-r--r-- 1 root root 428M Sep 13 21:17 rows.csv


<ipython-input-19-0f438d06faa8>:18: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sample = pd.read_csv('rows.csv', skiprows=lambda x: x > 0 and x % skip_ratio != 0)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100867 entries, 0 to 100866
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH DATE                     100867 non-null  object 
 1   CRASH TIME                     100867 non-null  object 
 2   BOROUGH                        69133 non-null   object 
 3   ZIP CODE                       69121 non-null   object 
 4   LATITUDE                       89073 non-null   float64
 5   LONGITUDE                      89073 non-null   float64
 6   LOCATION                       89073 non-null   object 
 7   ON STREET NAME                 79500 non-null   object 
 8   CROSS STREET NAME              62488 non-null   object 
 9   OFF STREET NAME                16982 non-null   object 
 10  NUMBER OF PERSONS INJURED      100866 non-null  float64
 11  NUMBER OF PERSONS KILLED       100865 non-null  float64
 12  NUMBER OF PEDESTRIANS INJURED 

<ipython-input-19-0f438d06faa8>:25: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nyc_df = pd.read_csv('rows.csv')


In [20]:
# number or rows

print(len(nyc_df))

# this is quite large so we will work with a sample while we experiment in pandas as least.

2118226


We'll take a random sample at 20% of the original data

In [21]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)

In [22]:
# we are also going to limit the columns to those we are going to work with

nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1',
                       'BOROUGH', 'VEHICLE TYPE CODE 1',
                       'NUMBER OF PERSONS INJURED']]

In [23]:
nyc_small.head(2)

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
430841,05/19/2019,Unspecified,QUEENS,Sedan,0.0
1779911,04/06/2013,Unspecified,MANHATTAN,SPORT UTILITY / STATION WAGON,0.0


Now, let's create the pyspark dataframe. Now we two frames with the same content
  * nyc_small: pandas
  * sdf_small: pyspark

In [24]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


# Lets check the schema quickly

print(sdf_small.schema)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


StructType([StructField('CRASH DATE', StringType(), True), StructField('CONTRIBUTING FACTOR VEHICLE 1', StringType(), True), StructField('BOROUGH', StringType(), True), StructField('VEHICLE TYPE CODE 1', StringType(), True), StructField('NUMBER OF PERSONS INJURED', StringType(), True)])


# Questions

Answer the following questions by porting the pandas code to the Spark API



# Question 1


> On what day do most crashes occcur?

In [25]:
# Pandas
nyc_small.groupby('CRASH DATE')['CRASH DATE'].count().sort_values(ascending=False).head(5)

CRASH DATE
11/15/2018    224
01/21/2014    217
12/15/2017    210
02/03/2014    205
12/14/2018    203
Name: CRASH DATE, dtype: int64

In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, to_date

spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

nyc_df = nyc_df.withColumn('CRASH DATE', to_date(col('CRASH DATE')))

result = (nyc_df.groupBy('CRASH DATE')
    .agg(count('*').alias('crash_count'))
        .orderBy(desc('crash_count'))
            .limit(5))

            result.show(truncate=False)

IndentationError: unexpected indent (<ipython-input-41-ab803808c577>, line 13)

# Question 2

> _Where do most crashes occur?_

In [33]:
nyc_small.groupby('BOROUGH')['BOROUGH'].count().sort_values(ascending=False).head(5)

BOROUGH
BROOKLYN         92591
QUEENS           78223
MANHATTAN        65192
BRONX            43322
STATEN ISLAND    12247
Name: BOROUGH, dtype: int64

In [42]:
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

nyc_df.createOrReplaceTempView("nyc_crashes")

query = """
SELECT BOROUGH, COUNT(*) as crash_count
FROM nyc_crashes
GROUP BY BOROUGH
ORDER BY crash_count DESC
LIMIT 5
"""

result = spark.sql(query)

result.show(truncate=False)

AttributeError: 'DataFrame' object has no attribute 'createOrReplaceTempView'

 # Question 3

 > What is the most common cause of accident in 'QUEENS'

In [35]:
nyc_small[(nyc_small.BOROUGH == 'QUEENS')]['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# you can also use a group by (to avoid the pandas value_counts function)

nyc_small[(nyc_small.BOROUGH == 'QUEENS')].groupby(
    'CONTRIBUTING FACTOR VEHICLE 1'
)['CONTRIBUTING FACTOR VEHICLE 1'].count().sort_values(ascending=False).head(5)

CONTRIBUTING FACTOR VEHICLE 1
Unspecified                       27318
Driver Inattention/Distraction    16548
Failure to Yield Right-of-Way      6763
Backing Unsafely                   3878
Following Too Closely              2922
Name: CONTRIBUTING FACTOR VEHICLE 1, dtype: int64

In [43]:
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

nyc_df.createOrReplaceTempView("nyc_crashes")

query = """
SELECT BOROUGH, COUNT(*) as crash_count
FROM nyc_crashes
GROUP BY BOROUGH
ORDER BY crash_count DESC
LIMIT 5
"""

result = spark.sql(query)

result.show(truncate=False)

AttributeError: 'DataFrame' object has no attribute 'createOrReplaceTempView'

# Question 4

> _What is the average number or injuries for specific cars driving in specific suburbs_


In [ ]:
nyc_small.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().sort_values(ascending=False).head(3)

In [44]:
## Spark?
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

nyc_df.createOrReplaceTempView("nyc_crashes")

query = """
SELECT `VEHICLE TYPE CODE 1`, BOROUGH, AVG(`NUMBER OF PERSONS INJURED`) as avg_injuries
FROM nyc_crashes
GROUP BY `VEHICLE TYPE CODE 1`, BOROUGH
ORDER BY avg_injuries DESC
LIMIT 3
"""

result = spark.sql(query)

result.show(truncate=False)

AttributeError: 'DataFrame' object has no attribute 'createOrReplaceTempView'